This notebook is a small exercise to practice exploring and visualizing data. 

**Objective:**

1. Is there any relationship between the Rotten Tomatoes ratings and those from IMDb?
2. Are there any insights to be gleaned from analyzing the lengths of the movies?
3. Is there a relationship between the number of enemies James Bond has killed and the user ratings of the movie in which they were killed?

### Install libraries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

### Load data

In [3]:
# We have already acquired the data, so we simply need to load it.
df = pd.read_csv('data/james_bond_data.csv').convert_dtypes()

In [4]:
# Take a look at the first few rows of the data.
df.head()

,Release,Movie,Bond,Bond_Car_MFG,US_Gross,World_Gross,Budget ($ 000s),Film_Length,Avg_User_IMDB,Avg_User_Rtn_Tom,Martinis,Kills_Bond
0,"June, 1962",Dr. No,Sean Connery,Sunbeam,"$16,067,035.00","$59,567,035.00","$1,000.00",110 mins,7.3,7.7,2,4
1,"August, 1963",From Russia with Love,Sean Connery,Bentley,"$24,800,000.00","$78,900,000.00","$2,000.00",115 mins,7.5,8.0,0,11
2,"May, 1964",Goldfinger,Sean Connery,Aston Martin,"$51,100,000.00","$124,900,000.00","$3,000.00",110 mins,7.8,8.4,1,9
3,"September, 1965",Thunderball,Sean Connery,Aston Martin,"$63,600,000.00","$141,200,000.00","$9,000.00",130 mins,7.0,6.8,0,20
4,"November, 1967",You Only Live Twice,Sean Connery,Toyota,"$43,100,000.00","$111,600,000.00","$9,500.00",117 mins,6.9,6.3,1,21


In [5]:
df.shape

(27, 12)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Release           27 non-null     string 
 1   Movie             27 non-null     string 
 2   Bond              27 non-null     string 
 3   Bond_Car_MFG      27 non-null     string 
 4   US_Gross          27 non-null     string 
 5   World_Gross       27 non-null     string 
 6   Budget ($ 000s)   27 non-null     string 
 7   Film_Length       27 non-null     string 
 8   Avg_User_IMDB     26 non-null     Float64
 9   Avg_User_Rtn_Tom  26 non-null     Float64
 10  Martinis          27 non-null     Int64  
 11  Kills_Bond        27 non-null     Int64  
dtypes: Float64(2), Int64(2), string(8)
memory usage: 2.8 KB


### Clean data

#### Rename columns

In [9]:
# Start by renaming the columns to make them easier to work with. Create a dictionary with new names.
new_column_names = {
   "Release": "release_date",
   "Movie": "movie_title",
   "Bond": "bond_actor",
   "Bond_Car_MFG": "car_manufacturer",
   "US_Gross": "income_usa",
   "World_Gross": "income_world",
   "Budget ($ 000s)": "movie_budget",
   "Film_Length": "film_length",
   "Avg_User_IMDB": "imdb",
   "Avg_User_Rtn_Tom": "rotten_tomatoes",
   "Martinis": "martinis_consumed",
   "Kills_Bond": "bond_kills",
   }

data = df.rename(columns=new_column_names)

In [10]:
data.columns

Index(['release_date', 'movie_title', 'bond_actor', 'car_manufacturer',
       'income_usa', 'income_world', 'movie_budget', 'film_length', 'imdb',
       'rotten_tomatoes', 'martinis_consumed', 'bond_kills'],
      dtype='object')

#### Identify and fix missing values

In [12]:
data.isna().sum()

release_date         0
movie_title          0
bond_actor           0
car_manufacturer     0
income_usa           0
income_world         0
movie_budget         0
film_length          0
imdb                 1
rotten_tomatoes      1
martinis_consumed    0
bond_kills           0
dtype: int64

In [15]:
# Where are the missing values?
data.loc[data.isna().any(axis="columns")] # any row with a missing value

,release_date,movie_title,bond_actor,car_manufacturer,income_usa,income_world,movie_budget,film_length,imdb,rotten_tomatoes,martinis_consumed,bond_kills
10,"April, 1977",The Spy Who Loved Me,Roger Moore,Lotus,"$46,800,000.00","$185,400,000.00","$14,000.00",125 mins,<NA>,<NA>,1,31


#### Fix data types

##### Financial columns

In [20]:
data[
    ["income_usa", "income_world", "movie_budget", "film_length"]
    ].head()

,income_usa,income_world,movie_budget,film_length
0,"$16,067,035.00","$59,567,035.00","$1,000.00",110 mins
1,"$24,800,000.00","$78,900,000.00","$2,000.00",115 mins
2,"$51,100,000.00","$124,900,000.00","$3,000.00",110 mins
3,"$63,600,000.00","$141,200,000.00","$9,000.00",130 mins
4,"$43,100,000.00","$111,600,000.00","$9,500.00",117 mins


In [49]:
# Add the missing values and convert the data of income_usa to Float64.
data = (
    df.rename(columns=new_column_names)
    .combine_first(
        pd.DataFrame({"imdb": {10: 7.1}, "rotten_tomatoes": {10: 6.8}}))
    .assign(income_usa=lambda data: (data["income_usa"]
                                     .replace("[$,]", "", regex=True)
                                     .astype("Float64")
                                        ),
            income_world=lambda data: (data["income_world"]
                                        .replace("[$,]", "", regex=True)
                                        .astype("Float64")
                                         ),
            movie_budget=lambda data: (data["movie_budget"]
                                        .replace("[$,]", "", regex=True)
                                        .astype("Float64")
                                       ),
            film_length=lambda data: (data["film_length"]
                                        .str.removesuffix("mins")
                                        .astype("Int64")
                                      ),
            release_date=lambda data: (pd.to_datetime(data["release_date"],
                                                        format="%B, %Y")
                                      ),
            release_year=lambda data: (data["release_date"]
                                       .dt.year
                                       .astype("Int64")),
                )
            )



In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 0 to 26
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   bond_actor         27 non-null     string        
 1   bond_kills         27 non-null     Int64         
 2   car_manufacturer   27 non-null     string        
 3   film_length        27 non-null     Int64         
 4   imdb               27 non-null     Float64       
 5   income_usa         27 non-null     Float64       
 6   income_world       27 non-null     Float64       
 7   martinis_consumed  27 non-null     Int64         
 8   movie_budget       27 non-null     Float64       
 9   movie_title        27 non-null     string        
 10  release_date       27 non-null     datetime64[ns]
 11  rotten_tomatoes    27 non-null     Float64       
 12  release_year       27 non-null     Int64         
dtypes: Float64(5), Int64(4), datetime64[ns](1), string(3)
memory usage: 3.2 

In [46]:
data.head()

,bond_actor,bond_kills,car_manufacturer,film_length,imdb,income_usa,income_world,martinis_consumed,movie_budget,movie_title,release_date,rotten_tomatoes
0,Sean Connery,4,Sunbeam,110,7.3,16067035.0,59567035.0,2,1000.0,Dr. No,"June, 1962",7.7
1,Sean Connery,11,Bentley,115,7.5,24800000.0,78900000.0,0,2000.0,From Russia with Love,"August, 1963",8.0
2,Sean Connery,9,Aston Martin,110,7.8,51100000.0,124900000.0,1,3000.0,Goldfinger,"May, 1964",8.4
3,Sean Connery,20,Aston Martin,130,7.0,63600000.0,141200000.0,0,9000.0,Thunderball,"September, 1965",6.8
4,Sean Connery,21,Toyota,117,6.9,43100000.0,111600000.0,1,9500.0,You Only Live Twice,"November, 1967",6.3
